In [1]:
'''
TODO:
    transformers:
        OneHotEncode columns
            sex(DONE), cabin(DONE), embarked(DONE)
        Figure out how to add these to a pipeline
        extract numerics from ticket to get families traveling together(DONE)
    END: create data processing pipeline
DONE:

'''
print('')

In [2]:
# https://www.kaggle.com/c/titanic/overview

import pandas as pd
import numpy as np

titanic_raw = pd.read_csv('./titanic_train.csv')
print(titanic_raw.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB
None


In [3]:
titanic_raw.head()

'''
columns to remove
    PassengerID
    Name
    Ticket
    ? Fare
categorical
    sex
    ? Cabin
    Embarked (C = Cherbourg, Q = Queenstown, S  = Southampton)
'''

'\ncolumns to remove\n    PassengerID\n    Name\n    Ticket\n    ? Fare\ncategorical\n    sex\n    ? Cabin\n    Embarked (C = Cherbourg, Q = Queenstown, S  = Southampton)\n'

In [4]:
# Let's define a custom Transformer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder


# Make OneHot encoding a column suck less
def OneHotSetup(X, onehotcol, dropfirst=True, droporiginal=False):
    # Create a new OneHotEncoder
    if dropfirst:
        dropfirst = 'first'
        hdr_offset = 1
    else:
        dropfirst = None
        hdr_offset = 0
        
    onehot = OneHotEncoder(sparse=False, drop=dropfirst) # remove 1st category
    
    # Fit our encoder
    X_onehot = onehot.fit(X[onehotcol].values.reshape(-1,1)) # Fit onehot to cabin_letter
    
    # Get a list of column names, minus the first one
    X_onehot_hdrs = X_onehot.categories_[0][hdr_offset:] # Take the list of categories minus the first one
    
    # Transform header to OG column name underscore category
    temp = []
    for val in X_onehot_hdrs:
        val = onehotcol + '_' + val
        temp.append(val)
    X_onehot_hdrs = temp
    
    # Create a DataFrame of new OneHot columns
    X_onehot_df = pd.DataFrame(onehot.transform(X[onehotcol].values.reshape(-1,1)))

    # Redefine the headers of the onehot cols
    X_onehot_df.columns = X_onehot_hdrs
    
    # Add new onehot columns to the original DF 
    X = pd.concat([X, X_onehot_df], axis=1)
    
    # Delete OneHot-ted column, if requested
    if droporiginal:
        X = X.drop([onehotcol], axis=1)
    
    return X

# Custom Transform class
class InitAttributeCleaner(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass
    def fit(self, X, y=None):
            return self # Nothing to do here, apparently
    def transform(self, X, y=None):
        # Remove unnecessary columns
        X = X.drop(['PassengerId', 'Name'],axis=1)
        
        # Handle the "Cabin" feature
        #     1. split into features "cabin_letter" and "cabin_number"
        #     2. replace nulls in both new columns with 'unreported'
        #     3. Use OneHotSetup to add onehotcolumns
        #     4. Replace NaN "cabin_number"s with 0, transform column to numeric
          #cabin_letter
        newcols = ['cabin_letter', 'cabin_number']                                 # New column names
        X[newcols] = X['Cabin'].str.extract(r'(\D*)(\d*)') # Split column
        X['cabin_letter'] = X[newcols].fillna('unreported')       # Replace nulls with 'unreported'
        X['cabin_letter'] = X[newcols].replace('', 'unreported')  # Replace nulls with 'unreported'
        X = OneHotSetup(X, 'cabin_letter', dropfirst=True, droporiginal=True)  # Call OneHotSetup
        X = X.drop(['cabin_letter_unreported'], axis=1)    # Remove cabin_letter_unreported (useless)
          #cabin_number
        X['cabin_number'] = X['cabin_number'].fillna(value=0) # Replace NA with 0
        X['cabin_number'] = X['cabin_number'].replace('', 0).astype('int64') # Replace '' with 0 and cast to int64
        X = X.drop(['Cabin'], axis=1)

        # Handle "sex" feature
        X = OneHotSetup(X, 'Sex', dropfirst=True, droporiginal=True)

        # Handle "Embarked" feature
        X['Embarked'] = X['Embarked'].fillna('U')  # Fill missing values with U for Unknown
        X = OneHotSetup(X, 'Embarked', dropfirst=False, droporiginal=False) # OneHotEncode it
        X = X.drop(['Embarked'], axis=1)   # Remove original category
        try:
            X = X.drop(['Embarked_U'], axis=1) # Remove unknown category, if necessary
        except:
            pass
        
        # Handle "Age" feature (we're going to impute a mean for missing values)
        X['Age'] = X['Age'].fillna(X['Age'].mean())

        # Handle "Ticket" column (remove characters, leave only numerics)
        X['ticket_number'] = X['Ticket'].str.extract(r'(?:.*? ){0,2}(\d*)') # Extract just the numeric at the end
        X['ticket_number'] = X['ticket_number'].replace('', 0).astype('int64') # These few "LINE" values that are missed by RE
        X = X.drop(['Ticket'], axis=1)
        
        self.final_columns = list(X.columns)
        
        return X

In [5]:
# Data Cleaning (Pipeline conversion)
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline


standardize_columns = ['Age', 'Fare', 'cabin_number', 'ticket_number']

columntransformer = ColumnTransformer([
                        ('standardizer', StandardScaler(), standardize_columns)
                ], remainder='passthrough',)

pipe = Pipeline([
                    ('init_clean', InitAttributeCleaner()),
                    ('col_trans', columntransformer)
                ])
'''
what a shitty to use API

The order of the columns in the transformed feature matrix follows the order of how the columns are specified in the
transformers list. Columns of the original feature matrix that are not specified are dropped from the resulting transformed
feature matrix, unless specified in the passthrough keyword. Those columns specified with passthrough are added at the right
to the output of the transformers.
'''
titanic_clean = pd.DataFrame(pipe.fit_transform(titanic_raw))

# transformed columns have the passthrough columns added after, so need to come up with new column order and rename
passthrough_columns = [x for x in pipe.named_steps.init_clean.final_columns if x not in standardize_columns]

titanic_clean.columns = standardize_columns + passthrough_columns

titanic_clean = titanic_clean.drop(['cabin_letter_T'], axis=1)

print(titanic_clean.columns.values)

['Age' 'Fare' 'cabin_number' 'ticket_number' 'Survived' 'Pclass' 'SibSp'
 'Parch' 'cabin_letter_B' 'cabin_letter_C' 'cabin_letter_D'
 'cabin_letter_E' 'cabin_letter_F' 'cabin_letter_F E' 'cabin_letter_F G'
 'cabin_letter_G' 'Sex_male' 'Embarked_C' 'Embarked_Q' 'Embarked_S']


In [6]:
# Time to split our test/train sets before going any further!

from sklearn.model_selection import StratifiedShuffleSplit

# Okay so first I need to split my X and y
X = titanic_clean.drop(['Survived'], axis=1)
y = titanic_clean['Survived']

# And now split using stratified shuffle split
splitter = StratifiedShuffleSplit(n_splits = 1, test_size = 0.2, random_state = 42)

for train_index, test_index in splitter.split(X, y):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

In [7]:
'''
Okay now I need to shortlist some models
I have no idea how to do this
I suppose this is a binary classifier soooo
we'll start off with the SGD Classifier
'''
from sklearn.linear_model import SGDClassifier
from sklearn.base import clone
from sklearn.model_selection import StratifiedKFold

sgd_clf = SGDClassifier(random_state=42)
skfolds = StratifiedKFold(n_splits=3, random_state=42, shuffle=True)

for train_index, test_index in skfolds.split(X_train, y_train):
#     print('train:', train_index, 'test', test_index)
    clone_clf = clone(sgd_clf)
    X_train_folds = X_train.iloc[train_index]
    y_train_folds = y_train.iloc[train_index]
    X_test_fold = X_train.iloc[test_index]
    y_test_fold = y_train.iloc[test_index]
    
    clone_clf.fit(X_train_folds, y_train_folds)
    y_pred = clone_clf.predict(X_test_fold)
    n_correct = sum(y_pred == y_test_fold)
    print(n_correct / len(y_pred))

0.726890756302521
0.7721518987341772
0.42616033755274263


In [8]:
from sklearn.model_selection import cross_val_score

sgd_scores = cross_val_score(sgd_clf, X_train, y_train, cv=5, scoring='accuracy')

In [9]:
class EveryoneDies(BaseEstimator):
    def fit(self, X, y=None):
        pass
    def predict(self, X):
        return np.zeros((len(X), 1), dtype=bool)

everyonedies_clf = EveryoneDies()
everysonedies_scores = cross_val_score(everyonedies_clf, X_train, y_train, cv=5, scoring='accuracy')

# Okay cool so I'm doing better than just guessing noone made it

In [10]:
# Alright let's try a RandomForestClassifier as well

from sklearn.ensemble import RandomForestClassifier

forest_clf = RandomForestClassifier(random_state=42)
forest_scores = cross_val_score(forest_clf, X_train, y_train, cv=10, scoring='accuracy')

In [11]:
# Let's score out a Guassian Naive Bayes
from sklearn.naive_bayes import GaussianNB

gnb_clf = GaussianNB()
gnb_scores = cross_val_score(gnb_clf, X_train, y_train, cv=5, scoring='accuracy')

In [12]:
# Let's try a logistic classifier
from sklearn.linear_model import LogisticRegression

lr_clf = LogisticRegression(multi_class='ovr')
lr_scores = cross_val_score(lr_clf, X_train, y_train, cv=10, scoring='accuracy')

In [13]:
print(f'SGD Classifier avg: {sgd_scores.mean()})')
print(f'EveryoneDies avg: {everysonedies_scores.mean()})')
print(f'RandomForestClassifier avg: {forest_scores.mean()})')
print(f'GaussianNB avg: {gnb_scores.mean()})')
print(f'SGD Classifier avg: {sgd_scores.mean()})')
print(f'LogisticRegression avg: {lr_scores.mean()})')

# So it looks like RandomForestClassifier is the best looking model so far!

SGD Classifier avg: 0.7599724219442529)
EveryoneDies avg: 0.6165468334482419)
RandomForestClassifier avg: 0.8330203442879499)
GaussianNB avg: 0.7219147050132966)
SGD Classifier avg: 0.7599724219442529)
LogisticRegression avg: 0.7977895148669797)


In [14]:
# quick test on forest classifier on test set?

forest_clf.fit(X_train, y_train)

aa = pd.DataFrame(forest_clf.predict(X_test)).value_counts()

correct = aa[0]
incorrect= aa[1]
print(f'percentage correct = {(correct/(incorrect+correct)*100)}')

# weak

percentage correct = 63.687150837988824


In [15]:
# Okay time to try tuning the forest

from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform, truncnorm, randint

forest_clf = RandomForestClassifier(random_state=42)

model_params = {"n_estimators" : randint(1, 200),
                "max_features" : truncnorm(a=0., b=1., loc=0.5, scale=0.01),
                "min_samples_split" : uniform(0.01, 0.199)
               }
               

rscv = RandomizedSearchCV(forest_clf, model_params, n_iter = 50, cv=5,
                          n_jobs = 2, scoring="accuracy")

rscv.fit(X_train, y_train)

print(rscv.best_score_)

0.8343346794051021


In [16]:
new_forest_clf = rscv.best_estimator_

# quick test on new forest classifier on test set
new_forest_clf.score(X_test, y_test)

test_results = new_forest_clf.predict(X_test)

In [17]:
# Okay let's get a final score from the competition test set!
titanic_test = pd.read_csv('./titanic_test.csv')
titanic_test.info()

# Store our Passenger IDs to use for our submission
test_set_ids = titanic_test['PassengerId']

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  418 non-null    int64  
 1   Pclass       418 non-null    int64  
 2   Name         418 non-null    object 
 3   Sex          418 non-null    object 
 4   Age          332 non-null    float64
 5   SibSp        418 non-null    int64  
 6   Parch        418 non-null    int64  
 7   Ticket       418 non-null    object 
 8   Fare         417 non-null    float64
 9   Cabin        91 non-null     object 
 10  Embarked     418 non-null    object 
dtypes: float64(2), int64(4), object(5)
memory usage: 36.0+ KB


In [18]:
'''
what a shitty to use API

The order of the columns in the transformed feature matrix follows the order of how the columns are specified in the
transformers list. Columns of the original feature matrix that are not specified are dropped from the resulting transformed
feature matrix, unless specified in the passthrough keyword. Those columns specified with passthrough are added at the right
to the output of the transformers.
'''
titanic_test_clean = pd.DataFrame(pipe.fit_transform(titanic_test))

# transformed columns have the passthrough columns added after, so need to come up with new column order and rename
passthrough_columns = [x for x in pipe.named_steps.init_clean.final_columns if x not in standardize_columns]

titanic_test_clean.columns = standardize_columns + passthrough_columns

titanic_test_clean['Fare'] = titanic_test_clean['Fare'].fillna(titanic_test_clean['Fare'].mean())

In [19]:
test_results = pd.DataFrame(new_forest_clf.predict(titanic_test_clean).astype(int))

test_results = pd.concat([test_set_ids, test_results], axis=1)

test_results.columns = ['PassengerId', 'survived']

test_results.to_csv('titanic_test_results.csv', index=False)

In [20]:
##############################################################
########## RE-CLEANING THE DATA FOR BETTER RESULTS! ##########
##############################################################

In [149]:
from sklearn.feature_extraction import FeatureHasher
from sklearn.preprocessing import MinMaxScaler
'''
Alright let's come up with a plan for EVERY SINGLE FEATURE
PassengerId
    DROP
Survived
    split into new vector, then DROP
Pclass
    these are 1, 2, or 3
        Should I -1 to get 0, 1, 2?
        Should I standardize/normalize this?
        Should I encode this column?
Name
    DROP
Sex
    male or female
        *onehotencode this guy, DROP one category and original column
Age
    177 NULL values
    from almost 0 to 80ish
        *let's fill in the NULL values with median/mean
        *then standardize/normalize
SibSp (# of siblins/spouses aboard the titanic: sibling = [step]brother/[step]sister, spource = husband/wife)
    I think I want to leave this one as categorical and not make it continous?
    this is an ORDINAL catergorical feature
        *Don't touch
Parch (# of parents/children aboard: mother, father, [step]daughter, [step]son. Nannoy does NOT count)
    this is another ORDINAL categorical feature
        *Don't touch
Ticket - 681 uniques, no nulls
    I'm not sure how to handle this
    Treat it categorically, but there are too many features for it to matter
        *.1: DROP
        *.2: Numericize, categorize(Onehot)
        *.3: Numericize, categorize(featurehashing)
Fare - Only 248 unique values wow (this has a very exponential shape hmm)
    *leave as continous
Cabin 687 nulls, 148 distinct values
    *Extract Deck (A, B, etc)
    *ordinally encode: A top deck, B below A, C below B, etc. Down to G
Embarked
    *onehotencode
'''

class InitAttributeCleaner2(BaseEstimator, TransformerMixin):
    def __init__(self, ticket_handle, hash_number=10):
        self.ticket_handle = ticket_handle
        self.hash_number = hash_number
    def fit(self, X, y=None):
            return self # Nothing to do here, apparently
    def transform(self, X, y=None):
        #PassengerId: DROP
        X = X.drop(['PassengerId'], axis=1)
        
        #Pclass: I don't want to categorize these, beacuse class 1 > 2 > 3 in SES (socioeconomic standing)
        #    So we're going to recenter from 1, 2, 3 to 0, 1, 2
        X['Pclass'] = X['Pclass'] - 1
        
        #Name: DROP
        X = X.drop(['Name'], axis=1)
        
        #Sex: OneHotEncode this guy, then remove the original and female column
        X = pd.concat([X, pd.get_dummies(X['Sex'])], axis=1)
        X = X.drop(['Sex', 'female'], axis=1) # I could use "drop_first" param above for 'female' but w/e
        
        #Age: Fill NULLs with median, then standardize or normalize
        X['Age'] = X['Age'].fillna(X['Age'].median())
        
        min_max_scaler = MinMaxScaler()
        X['Age'] = min_max_scaler.fit_transform(np.array(X['Age']).reshape(-1,1))
        
        
        if(self.ticket_handle == 'drop'):
            pass
            
        elif(self.ticket_handle == 'numeric'):
            X['ticket_number'] = X['Ticket'].str.extract(r'(?:.*? ){0,2}(\d*)') # Extract just the numeric at the end
            X['ticket_number'] = X['ticket_number'].replace('', 0).astype('int64') # These few "LINE" values that are missed by RE
            
        elif(self.ticket_handle == 'onehot'):
            X['ticket_number'] = X['Ticket'].str.extract(r'(?:.*? ){0,2}(\d*)') # Extract just the numeric at the end
            X['ticket_number'] = X['ticket_number'].replace('', 0).astype('int64') # These few "LINE" values that are missed by RE
            X = pd.concat([X, pd.get_dummies(X['ticket_number'], drop_first=True, prefix='ticket_num')], axis=1)
            X = X.drop(['ticket_number'], axis=1)
            
        elif(self.ticket_handle == 'hash'):
            hasher = FeatureHasher(n_features=self.hash_number, input_type='string') # Create our hasher

            hash_cols = [] # Populate a list so we can have column names
            for i in range(0, self.hash_number):
                hash_cols.append(f'ticket_hash_{i}')
            # send it
            hashed_cat = hasher.fit_transform(X['Ticket']).todense()
            hashed_cat = pd.DataFrame(hashed_cat, columns=hash_cols)
            
            # This is necessary for god knows why
            X.reset_index(drop=True, inplace=True)
            hashed_cat.reset_index(drop=True, inplace=True)

            X = pd.concat([X, hashed_cat], axis=1)
            
        else:
            raise RuntimeError('invalid handler for Ticket. Try drop, numeric, onehot, or hash')

        X = X.drop(['Ticket'], axis=1)
        
        
        #Cabin: Extract Deck, ordinally encode
        X['deck'] = X['Cabin'].str.extract(r'(\D*)\d*') # Split column
        X['deck'] = X['deck'].fillna('U')

        mapper = {'A' : 0,
                  'B' : 1,
                  'C' : 2,
                  'D' : 3,
                  'E' : 4,
                  'F E' : 5,
                  'F' : 6,
                  'F G' : 7,
                  'G' : 8,
                  'T' : 9,
                  'U' : 10
                }
        X['deck'] = X['deck'].replace(mapper)
        X = X.drop(['Cabin'], axis=1)
        
        
        #Embarked: OneHotEncode this bad boy
        X = pd.concat([X, pd.get_dummies(X['Embarked'], prefix='embarked')], axis=1)
        X = X.drop(['Embarked'], axis=1)
        
        return X


In [146]:
# Looks like I've handled ever feature! Time to send it!
X = titanic_raw.drop(['Survived'], axis=1)
y = titanic_raw.loc[:, 'Survived']

splitter = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)

for train_index, test_index in splitter.split(X, y):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

In [118]:
# newcols = ['cabin_letter', 'cabin_number']                                 # New column names
# X[newcols] = X['Cabin'].str.extract(r'(\D*)(\d*)') # Split column
X = titanic_raw

# X = X.drop(['Sex', 'female'], axis=1) # I could use "drop_first" param above for 'female' but w/e
 

In [153]:
# Let's create the pipeline we're going to use with GridSearchCV
# We'll create 1 X_train for each type of transformer ticket handles first

rf_clf = RandomForestClassifier()

model_params = {'n_estimators' : randint(4, 400),
                'max_features' : truncnorm(a=0., b=1., loc=0.5, scale=0.1),
                'min_samples_split' : uniform(0.01, 0.5)
               }

ticket_clean_types = ['drop', 'numeric', 'onehot', 'hash']
ticket_clean_best_estimators = dict((el,0) for el in ticket_clean_types)

# Alright we want to find the most accurate estimator for the 4 different ways I came up with to handle "Ticket" feature
for cleantype in ticket_clean_types:
    print(f'cleantype: {cleantype}')
    # Reset X_train
    X = titanic_raw.drop(['Survived'], axis=1)
    y = titanic_raw.loc[:, 'Survived']

    splitter = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)

    for train_index, test_index in splitter.split(X, y):
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    
    rf_clf = RandomForestClassifier()
    
    # Clean X_train
    cleaner = InitAttributeCleaner2(ticket_handle=cleantype, hash_number=10)

    X_train = cleaner.fit_transform(X_train)

    # Fit, print best params and score, store the best estimator
    rscv = RandomizedSearchCV(rf_clf, model_params, n_iter=300, cv=5, n_jobs=4, scoring='accuracy')

    rscv.fit(X_train, y_train)
    
    ticket_clean_best_estimators[cleantype] = rscv.best_estimator_
    print(f'tickethandle: {cleantype}, best score: {rscv.best_score_}, best params: {rscv.best_params_}')
    print('')

cleantype: drop
tickethandle: drop, best score: 0.827351521717719, best params: {'max_features': 0.5518781082566893, 'min_samples_split': 0.025844306118618703, 'n_estimators': 118}

cleantype: numeric
tickethandle: numeric, best score: 0.8399783315276274, best params: {'max_features': 0.5111487598136749, 'min_samples_split': 0.0116647394336059, 'n_estimators': 223}

cleantype: onehot
tickethandle: onehot, best score: 0.8385797301290259, best params: {'max_features': 0.5174243960252174, 'min_samples_split': 0.011261021770732618, 'n_estimators': 291}

cleantype: hash
tickethandle: hash, best score: 0.8399290849995076, best params: {'max_features': 0.5369024826852938, 'min_samples_split': 0.010901775287247553, 'n_estimators': 327}



In [ ]:
# We're going to go with the Random Forest since it did best last time
# We're going to use the GridSearchCV:
#    Not randomized because I want to check out the differnent hyperparameters in the transformer